In [1]:
from pathlib import Path
import pandas as pd
pd.set_option('display.max_columns', None) # Display all columns
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
raw_data_dir = Path(Path.cwd().parent, "data", "raw")
input_fname = Path(raw_data_dir, "dataset.csv")
df = pd.read_csv(input_fname, low_memory=False)

# Tran Test Split
Before I undertake any data cleaning or transformation I'll put aside a test set for final model performance. 
I intend to use cross validation during training to evaluate the model iteratively, so don't need to set aside a validation set at this point. 

Due to the high imbalance present in the target feature, seen in the previous notebook, it is important the train test split is stratified. 

In [3]:
perc_pos_prior = 100*(df['CHURN'].value_counts()[1] / len(df)) # % positive target feature prior to splitting
perc_pos_prior

0.67

In [4]:
X = df.drop('CHURN', axis = 1)
y = df['CHURN']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify = y) # 80:20 train:test split

In [5]:
y_train_unique, y_train_counts = np.unique(y_train, return_counts=True)
y_test_unique, y_test_counts = np.unique(y_test, return_counts=True)

perc_pos_post_train = y_train_counts[1]/ (sum(y_train_counts)) * 100
perc_pos_post_test = y_train_counts[1]/ (sum(y_train_counts)) * 100

print(f"% of positive target features in y_train: {perc_pos_post_train } %")
print(f"% of positive target features in y_train: {perc_pos_post_test} %")

assert np.isclose(a = perc_pos_post_train,b =  perc_pos_prior)
assert np.isclose(a = perc_pos_post_test,b =  perc_pos_prior)


% of positive target features in y_train: 0.67 %
% of positive target features in y_train: 0.67 %


Now we have a stratified train test split we can put aside the test set for future use. 

# Cleaning and Transformation

In [6]:
X.head()

,SALESFORCEACCOUNTID,ACCOUNTING_MONTH,RENEWAL_MONTH,CONTRACT_START_DATE,REGION,SECTOR,TENURE_MONTHS,DNB_GLOBAL_SALES_REVENUE,DNB_GLOBAL_EMPLOYEE_COUNT,CONTRACT_LENGTH,ARR,SEATS,COMPETITOR_SEATS,DETECTEDSEATSCOUNT,PRODUCT_ONE,PRODUCT_TWO,PRODUCT_THREE,PRODUCT_FOUR,PRODUCT_FIVE,PRODUCT_SIX,PRODUCT_SEVEN,PRODUCT_EIGHT,PRODUCT_NINE,PRODUCT_TEN,PRODUCT_ELEVEN,PRODUCT_TWELVE,PRODUCT_THIRTEEN,PRODUCT_FOURTEEN,PRODUCT_FIFTEEN,PRODUCT_SIXTEEN,PRODUCT_SEVENTEEN,PRODUCT_EIGHTEEN,PRODUCT_NINETEEN,PRODUCT_TWENTY,PRODUCT_TWENTYONE,PRODUCT_TWENTYTWO,PRODUCT_TWENTYTHREE,LICENSINGSPECIALIST_CHANGE,SALESREP_CHANGE,UPSELLMANAGER_CHANGE,ACCOUNTMANAGER_CHANGE,CHURN_RISK_DISCUSSION,CUSTOMER_HEALTH_CHECK,CS_EVENT_ATTENDED,CROSS_SELL_RECENCY,SEATS_DOWNSELL_RECENCY,PRODUCT_DOWNSELL_RECENCY,SEATS_UPSELL_RECENCY,PCT_PRODUCT_TWO_ENABLED,PCT_PRODUCT_THREE_ENABLED,PCT_PRODUCT_FOUR_ENABLED,PCT_PRODUCT_FIVE_ENABLED,PCT_PRODUCT_SIX_ENABLED,PCT_PRODUCT_NINE_ENABLED,PCT_PRODUCT_TWO_BEST_PRACTICE,PCT_PRODUCT_THREE_BEST_PRACTICE,PCT_PRODUCT_FOUR_BEST_PRACTICE,CUSTOMER_BEHAVIOUR_ONE,CUSTOMER_BEHAVIOUR_TWO,CUSTOMER_BEHAVIOUR_THREE,CUSTOMER_BEHAVIOUR_FOUR,CUSTOMER_BEHAVIOUR_FIVE,CUSTOMER_BEHAVIOUR_SIX,CUSTOMER_BEHAVIOUR_SEVEN,CUSTOMER_BEHAVIOUR_EIGHT,CUSTOMER_BEHAVIOUR_NINE,CUSTOMER_BEHAVIOUR_TEN,CUSTOMER_BEHAVIOUR_ELEVEN,MAX_SUPPORT_CASE_DAYSTOCLOSE,MAX_SUPPORT_CASE_TIMETOFIRSTRESPONSE,AVG_SUPPORT_CASE_PRIORITY_SCORE,SUM_SEVERE_CASES,SUM_HIGH_CASES,SUM_MEDIUM_CASES,SUM_LOW_CASES,SUM_STANDARD_CASES,SUPPORT_CASE_NUMBEROFSLABREACHES,BACKLOG,SURVEY_AVG_CXI_SCORE,SURVEY_AVG_NPS_SCORE,SURVEY_AVG_CASE_MOOD_SCORE
0,0011N00001hBZM7QAO,2021-10-01 00:00:00+00:00,2021-12-01T00:00:00Z,2020-12-24T00:00:00Z,UKI,Professional Services,126,NaN,NaN,364,27241.17,330,0,1.015152,1,1,1,1,1,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9,21,6,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,1,NaN,NaN,NaN
1,0011N00001hBZM7QAO,2021-11-01 00:00:00+00:00,2021-12-01T00:00:00Z,2020-12-24T00:00:00Z,UKI,Professional Services,127,NaN,NaN,364,27241.17,330,0,1.021212,1,1,1,1,1,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10,22,7,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,1,NaN,NaN,NaN
2,0011N00001hBZM7QAO,2021-12-01 00:00:00+00:00,2022-12-01T00:00:00Z,2021-12-24T00:00:00Z,UKI,Professional Services,128,NaN,NaN,364,30349.76,350,0,0.940000,1,1,1,1,1,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,11,23,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,1,NaN,NaN,NaN
3,0011N00001hBZM7QAO,2022-01-01 00:00:00+00:00,2022-12-01T00:00:00Z,2021-12-24T00:00:00Z,UKI,Professional Services,129,NaN,NaN,364,30349.76,350,0,0.960000,1,1,1,1,1,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,24,1,0,0,0,0,0,0,0,0,0,0,116598,0,0,0,11525,362,4,14,1892,0,0,0,0.0,0,0,0,0,0,0,1,NaN,NaN,NaN
4,0011N00001hBZM7QAO,2022-02-01 00:00:00+00:00,2022-12-01T00:00:00Z,2021-12-24T00:00:00Z,UKI,Professional Services,130,NaN,NaN,364,30349.76,350,0,0.940000,1,1,1,1,1,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,25,2,0,0,0,0,0,0,0,0,0,0,40704,0,0,0,3804,194,17,4,737,0,0,0,0.0,0,0,0,0,0,0,1,NaN,NaN,NaN


In [7]:
X.columns

Index(['SALESFORCEACCOUNTID', 'ACCOUNTING_MONTH', 'RENEWAL_MONTH',
       'CONTRACT_START_DATE', 'REGION', 'SECTOR', 'TENURE_MONTHS',
       'DNB_GLOBAL_SALES_REVENUE', 'DNB_GLOBAL_EMPLOYEE_COUNT',
       'CONTRACT_LENGTH', 'ARR', 'SEATS', 'COMPETITOR_SEATS',
       'DETECTEDSEATSCOUNT', 'PRODUCT_ONE', 'PRODUCT_TWO', 'PRODUCT_THREE',
       'PRODUCT_FOUR', 'PRODUCT_FIVE', 'PRODUCT_SIX', 'PRODUCT_SEVEN',
       'PRODUCT_EIGHT', 'PRODUCT_NINE', 'PRODUCT_TEN', 'PRODUCT_ELEVEN',
       'PRODUCT_TWELVE', 'PRODUCT_THIRTEEN', 'PRODUCT_FOURTEEN',
       'PRODUCT_FIFTEEN', 'PRODUCT_SIXTEEN', 'PRODUCT_SEVENTEEN',
       'PRODUCT_EIGHTEEN', 'PRODUCT_NINETEEN', 'PRODUCT_TWENTY',
       'PRODUCT_TWENTYONE', 'PRODUCT_TWENTYTWO', 'PRODUCT_TWENTYTHREE',
       'LICENSINGSPECIALIST_CHANGE', 'SALESREP_CHANGE', 'UPSELLMANAGER_CHANGE',
       'ACCOUNTMANAGER_CHANGE', 'CHURN_RISK_DISCUSSION',
       'CUSTOMER_HEALTH_CHECK', 'CS_EVENT_ATTENDED', 'CROSS_SELL_RECENCY',
       'SEATS_DOWNSELL_RECENCY', 'PRODUC